In [1]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

import plotly.express as px
from plotly.subplots import make_subplots
import plotly.graph_objects as go

import os

import pickle

from tqdm import tqdm

import lightgbm
import xgboost
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import AdaBoostClassifier 

import catboost
import gc
import random
random.seed(2233)

# for ens X
from sklearn.metrics import average_precision_score 
from sklearn.model_selection import GroupKFold
from sklearn.model_selection import StratifiedGroupKFold
from sklearn.model_selection import StratifiedShuffleSplit

import matplotlib.pyplot as plt

In [2]:
MODEL = "xgboost"
all_cols = ['target', 'Visit', 'AccV_max_past_20_maxs', 'AccV_min_past_10_maxs', 'AccML_mean_past_10_mins', 'AccML_mean_past_20_mins', 'graph_first_half_AccV_max', 'AccML_std_past_20_mins', 'AccV_mean_past_20_maxs', 'precent_prograss_moving_mean_all_AccAP', 'AccAP_pct', 'AccV_min_past_20_mins', 'precent_prograss_Time', 'AccV_mean_past_20_mins', 'moving_std_AccV_pct_10', 'fence_AccAP_pct_m10_w10_past', 'graph_second_half_AccV_max', 'Subject', 'AccV_mean_past_10_mins', 'precent_prograss_AccAP', 'fence_AccAP_pct_m35_w5_past', 'AccML_min_past_10_maxs', 'graph_first_half_AccV_mean', 'AccML_min_past_20_maxs', 'fence_AccAP_pct_m15_w10_past', 'graph_second_half_AccV_std', 'AccV_std_past_20_maxs', 'AccV_min_past_10_mins', 'fence_AccAP_pct_m10_w5_past', 'precent_prograss_moving_std_all_AccAP', 'AccV_min_past_20_maxs', 'graph_AccV_min', 'AccAP_std_past_20_maxs', 'graph_second_half_AccV_min', 'AccML_std_past_10_mins', 'fence_AccAP_pct_m5_w10_past', 'AccV_std_past_10_maxs', 'precent_prograss_moving_std_all_AccV', 'AccAP_std_past_20_mins', 'AccAP_std_past_10_mins', 'graph_AccV_max', 'AccV_max_past_10_maxs', 'moving_std_AccV_pct_5', 'graph_first_half_AccV_min', 'fence_AccAP_pct_m35_w10_past', 'AccAP_min_past_10_maxs', 'fence_AccAP_pct_m25_w5_past', 'AccML_mean_past_20_maxs', 'AccAP_std_past_10_maxs', 'graph_second_half_AccV_mean', 'AccML_std_past_10_maxs', 'fence_AccAP_pct_m25_w10_past', 'precent_prograss_moving_mean_all_AccV', 'moving_std_all_AccAP', 'AccV_mean_past_10_maxs', 'AccAP_min_past_20_maxs', 'precent_prograss_AccV', 'id', 'AccV_max_past_20_mins', 'AccV_max_past_10_mins', 'AccML_std_past_20_maxs', 'graph_first_half_AccV_std', 'AccML_max_past_30_maxs', 'AccML_max_past_40_maxs', 'AccAP_max_past_30_maxs', 'AccAP_max_past_40_maxs', 'AccAP_max_past_30_mins', 'AccAP_max_past_40_mins', 'graph_AccAP_mean', 'graph_AccAP_std', 'graph_AccAP_min', 'graph_AccAP_max']
parms = {"verbosity": 2,'n_estimators': 96, 'max_depth': 8, 'learning_rate': 0.09978972217945757, 'min_child_weight': 1, 'subsample': 0.13028595951808125, 'gamma': 0.5886903158423963, 'colsample_bytree': 0.9886044974665293, 'reg_alpha': 0.8251953840806328, 'reg_lambda': 0.9903469193310442}
de_train = pd.read_pickle("/kaggle/input/make-data-v9-de/train_de.pkl")[all_cols]
td_train = pd.read_pickle("/kaggle/input/make-data-v9-td/train_td.pkl")[all_cols]

de_valid = pd.read_pickle("/kaggle/input/make-data-v9-de/valid_de.pkl")[all_cols]
td_valid = pd.read_pickle("/kaggle/input/make-data-v9-td/valid_td.pkl")[all_cols]

td_train["Valid"] = 1
td_train["Task"] = 1

td_valid["Valid"] = 1
td_valid["Task"] = 1

df_for_cv =  de_train.append(td_train).reset_index(drop=True)

## ATTENTION!! DOWN SAMPLING HERE!
df_for_sub = df_for_cv.append(de_valid).append(td_valid).sample(frac=0.60).sort_index().reset_index(drop=True)

# folds_train_indexes_cv = pickle.load(open("/kaggle/input/v9-make-folds/folds_train_indexs_cv.pkl","rb"))
# folds_train_indexes_full = pickle.load(open("/kaggle/input/v9-make-folds/folds_train_indexs_full.pkl","rb"))

# folds_valid_indexes_cv = pickle.load(open("/kaggle/input/v9-make-folds/folds_valid_indexs_cv.pkl","rb"))
# folds_valid_indexes_full = pickle.load(open("/kaggle/input/v9-make-folds/folds_valid_indexs_full.pkl","rb"))

In [3]:
# # angry work-around
# def angry_workaround(df, indexs):
#     for fold in range(len(indexs)):
#         df.loc[indexs[fold][0], "target"] = 0    
#         df.loc[indexs[fold][1], "target"] = 1 
#         df.loc[indexs[fold][2], "target"] = 2    
#         df.loc[indexs[fold][3], "target"] = 3
#     return df


# df_for_cv = angry_workaround(df_for_cv, folds_train_indexes_cv)
# df_for_sub = angry_workaround(df_for_sub, folds_train_indexes_full)

In [4]:
# folds_train_indexs_full = pickle.load(open("/kaggle/input/v9-make-folds/folds_train_indexs_full.pkl", "rb"))
# train_ind = set([])
# for ele in folds_train_indexs_full:
#     for ind in ele:
#         train_ind.add(ind)
# relevant_ind = list(train_ind)

# df_for_sub = df_for_sub.loc[relevant_ind]

In [5]:
def get_model(parms= None, model_name="lgbm"):
    if model_name == "lgbm":
        if parms is None:
            model = lightgbm.LGBMClassifier()
        else:
            model = lightgbm.LGBMClassifier(**parms)
    elif model_name == "rf":
        if parms is None:
            model = RandomForestClassifier()
        else:
            model = RandomForestClassifier(**parms)     
            
    elif model_name == "xgboost":
        if parms is None:
            model = xgboost.XGBClassifier()
        else:
            model = xgboost.XGBClassifier(**parms) 
            
    elif model_name == "catboost":
        if parms is None:
            model = catboost.CatBoostClassifier()
        else:
            model = catboost.CatBoostClassifier(**parms) 
            
    elif model_name == "adaboost":
        if parms is None:
            model = AdaBoostClassifier()
        else:
            model = AdaBoostClassifier(**parms)
            
    return model

In [6]:
def fix_y(y):
    return y - 1 # (1,2,3) -> (0,1,2)

def fit_save(train_X,train_y, train_cols, cv_or_full,parms= None, model_name="lgbm"):
    no_train_cols = ["Time","Valid","Task",'Subject','file', 'id', 'target']    
    train_cols = [c for c in train_cols if c not in no_train_cols]
    
    model = get_model(parms, model_name)
    no_sh = False
    
    if set(train_y.unique()) == set([1,2,3]): # no SH
        train_y = fix_y(train_y)
        no_sh = True
    gc.collect()
    model.fit(train_X[train_cols], train_y)
    gc.collect()
    model_dict = {"model_name" : model_name,
                 "model": model,
                 "parms": parms,
                 "features": train_cols}
    pickle.dump(model_dict, open(f"model_dict_{model_name}_{cv_or_full}.pkl", "wb"))

In [7]:
#print(f"fit_save_{MODEL}_for_CV")
#fit_save(df_for_cv,df_for_cv["target"], train_cols=all_cols, parms= parms, model_name=MODEL, cv_or_full = "cv")

# print(f"Make_ENS_Xy_{MODEL}_for_CV")
# preds, gt = pipeline(df_for_cv, all_cols, folds=5, parms=parms, model_name=MODEL, cv_or_full = "cv")

In [8]:
print(f"fit_save_{MODEL}_for_submission")
fit_save(df_for_sub,df_for_sub["target"], train_cols=all_cols, parms= parms, model_name=MODEL, cv_or_full = "full")

# print(f"Make_ENS_Xy_{MODEL}_for_submission")
# preds, gt = pipeline(df_for_sub, all_cols, folds_total=folds_total, folds_start=folds_start,folds_end=folds_end,
#                      train_folds_list=folds_train_indexes_full , valid_folds_list= folds_valid_indexes_full, parms=parms, model_name=MODEL, cv_or_full = "full")


fit_save_xgboost_for_submission
[04:50:00] INFO: ../src/gbm/gbtree.cc:147: Tree method is automatically selected to be 'approx' for faster speed. To use old behavior (exact greedy algorithm on single machine), set tree_method to 'exact'.
[04:50:01] INFO: ../src/data/simple_dmatrix.cc:102: Generating new Gradient Index.
[04:51:06] INFO: ../src/data/simple_dmatrix.cc:102: Generating new Gradient Index.
[04:51:33] INFO: ../src/data/simple_dmatrix.cc:102: Generating new Gradient Index.
[04:51:59] INFO: ../src/data/simple_dmatrix.cc:102: Generating new Gradient Index.
[04:52:27] INFO: ../src/data/simple_dmatrix.cc:102: Generating new Gradient Index.
[04:52:52] INFO: ../src/data/simple_dmatrix.cc:102: Generating new Gradient Index.
[04:53:20] INFO: ../src/data/simple_dmatrix.cc:102: Generating new Gradient Index.
[04:53:46] INFO: ../src/data/simple_dmatrix.cc:102: Generating new Gradient Index.
[04:54:14] INFO: ../src/data/simple_dmatrix.cc:102: Generating new Gradient Index.
[04:54:39] INFO